In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import files
uploaded = files.upload()

RUN 1

In [ ]:
# ----- STEP 0: Mount Google Drive (if not already mounted) -----
from google.colab import drive
drive.mount('/content/drive')

# ----- STEP 1: Import Libraries and Set Mixed Precision Policy -----
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB3  # Or change to EfficientNetB4/DenseNet121 as desired
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model

# Use the new mixed precision API
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# ----- STEP 2: Define Directory Paths and Load CSVs -----
# Update these paths to match your Google Drive structure
train_img_dir = '/content/drive/My Drive/aptos2019-blindness-detection/train_images/'
test_img_dir  = '/content/drive/My Drive/aptos2019-blindness-detection/test_images/'

train_csv_path = '/content/train.csv'
test_csv_path  = '/content/test.csv'

# Load CSV files
train_df = pd.read_csv(train_csv_path)
test_df  = pd.read_csv(test_csv_path)


# Convert 'diagnosis' to string (required for categorical generators)
train_df['diagnosis'] = train_df['diagnosis'].astype(str)

# Create a filename column by appending the '.png' extension to id_code
train_df['filename'] = train_df['id_code'] + '.png'
test_df['filename']  = test_df['id_code'] + '.png'



# ----- STEP 3: Split Data into Training and Validation Sets -----
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
# Ensure filename and diagnosis columns are properly set
train_df['filename'] = train_df['id_code'] + '.png'
val_df['filename']   = val_df['id_code'] + '.png'
train_df['diagnosis'] = train_df['diagnosis'].astype(str)
val_df['diagnosis']   = val_df['diagnosis'].astype(str)

# ----- STEP 4: Define Image Size and Create Data Generators with Enhanced Augmentation -----
# Image size is optimized to 380x380 for EfficientNetB3; adjust if needed
IMAGE_SIZE = (380, 380)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,               # Smaller rotation for medical images
    brightness_range=[0.8, 1.2],       # Adjust brightness/contrast relevant for retinal scans
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='constant',
    cval=0                         # Black background for retinal images
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_img_dir,
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    validate_filenames=True
)

# Note: For validation, if the images are in the same folder as training images, use train_img_dir.
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_img_dir,
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

# ----- STEP 5: Set Up Callbacks (Learning Rate Scheduler, EarlyStopping, and ModelCheckpoint) -----
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True),
    reduce_lr
]

# ----- STEP 6: Build the Model Using a Pretrained Base (EfficientNetB3) and Custom Top Layers -----
# Load base model without top layers, with pretrained ImageNet weights
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
base_model.trainable = False  # Freeze the base model initially

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)  # Batch Normalization re-calibration
# The final layer: note 'dtype' is set to float32 to counter mixed precision output issues
output = Dense(5, activation='softmax', dtype='float32')(x)

model = Model(inputs=base_model.input, outputs=output)

# Show model architecture
model.summary()

# ----- STEP 7: Compile the Model with Additional Evaluation Metrics -----
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',  # Optionally, use a custom weighted or focal loss for imbalanced classes
    metrics=['accuracy', AUC(), Precision(), Recall()]
)

# ----- (Optional) STEP 8: Progressive Unfreezing for Fine-Tuning -----
# After initial training, you can gradually unfreeze layers for controlled fine-tuning.
# Uncomment the following block after initial training and recompile with a lower learning rate.
'''
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy', AUC(), Precision(), Recall()]
)
'''

# ----- STEP 9: Train the Model -----
history = model.fit(
    train_generator,
    epochs=10,  # Adjust epochs as needed
    validation_data=val_generator,
    callbacks=callbacks
)

# ----- STEP 10: Implement Grad-CAM for Explainability -----
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """
    Generates a Grad-CAM heatmap for a given image and model.
    """
    # Create a model that maps the input image to the activations of the last conv layer and predictions
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Compute the gradient of the predicted class (or the provided pred_index) with respect to the activations
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    # Compute the gradients of the target class with respect to the convolutional layer outputs
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel in the feature map array by "how important this channel is" with regard to the target class
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize the heatmap between 0 and 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, alpha=0.4):
    """
    Overlays the Grad-CAM heatmap on the original image.
    """
    # Load the original image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Resize heatmap to match image dimensions
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)

    # Apply a color map to the heatmap
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Superimpose the heatmap on the image
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)

    plt.figure(figsize=(8, 8))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# ----- STEP 11: Demonstrate Grad-CAM on a Sample Image -----
# Choose a sample image from the training directory
sample_img_path = os.path.join(train_img_dir, train_df['filename'].iloc[0])
# Preprocess the image
from tensorflow.keras.preprocessing import image
img = image.load_img(sample_img_path, target_size=IMAGE_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Replace 'top_conv' with the actual name of your last convolutional layer in EfficientNetB3
# You can find the name by inspecting model.summary()
last_conv_layer_name = None
for layer in base_model.layers[::-1]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer_name = layer.name
        break
if last_conv_layer_name is None:
    raise ValueError("No convolutional layer found in the base model.")

# Generate the Grad-CAM heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
# Display the heatmap overlaid on the original image
display_gradcam(sample_img_path, heatmap)


RUN 2

In [ ]:
# ----- STEP 0: Mount Google Drive -----
from google.colab import drive
drive.mount('/content/drive')

# ----- STEP 1: Import Libraries and Set Mixed Precision Policy -----
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB4  # Using EfficientNetB4 for higher capacity
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.mixed_precision import set_global_policy

# Enable mixed precision training for speed on compatible GPUs
set_global_policy('mixed_float16')

# ----- STEP 2: Define Directory Paths and Load CSVs -----
train_img_dir = '/content/drive/My Drive/aptos2019-blindness-detection/train_images/'
test_img_dir  = '/content/drive/My Drive/aptos2019-blindness-detection/test_images/'
train_csv_path = '/content/drive/My Drive/aptos2019-blindness-detection/train.csv'
test_csv_path  = '/content/drive/My Drive/aptos2019-blindness-detection/test.csv'

train_df = pd.read_csv(train_csv_path)
test_df  = pd.read_csv(test_csv_path)

train_df['diagnosis'] = train_df['diagnosis'].astype(str)
train_df['filename'] = train_df['id_code'] + '.png'
test_df['filename']  = test_df['id_code'] + '.png'

# ----- STEP 3: Split Data into Training and Validation Sets -----
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
train_df['filename'] = train_df['id_code'] + '.png'
val_df['filename']   = val_df['id_code'] + '.png'
train_df['diagnosis'] = train_df['diagnosis'].astype(str)
val_df['diagnosis']   = val_df['diagnosis'].astype(str)

# ----- STEP 4: Define Image Size and Create Data Generators -----
IMAGE_SIZE = (380, 380)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='constant',
    cval=0
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_img_dir,
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    validate_filenames=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_img_dir,  # Assuming validation images are in the same folder
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

# ----- STEP 5: Set Up Callbacks -----
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True),
    reduce_lr
]

# ----- STEP 6: Build the Model Using EfficientNetB4 and Unfreeze All Layers -----
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
base_model.trainable = True  # Unfreeze all layers for full fine-tuning

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
output = Dense(5, activation='softmax', dtype='float32')(x)

model = Model(inputs=base_model.input, outputs=output)
model.summary()

# ----- STEP 7: Compile the Model with a Lower Learning Rate for Fine-Tuning -----
model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower LR for full fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy', AUC(), Precision(), Recall()]
)

# ----- STEP 8: Train the Model (Full Fine-Tuning) -----
history = model.fit(
    train_generator,
    epochs=20,  # Increase epochs as needed
    validation_data=val_generator,
    callbacks=callbacks
)

# ----- STEP 9: Grad-CAM for Explainability -----
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    plt.figure(figsize=(8, 8))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# ----- STEP 10: Demonstrate Grad-CAM on a Sample Image -----
sample_img_path = os.path.join(train_img_dir, train_df['filename'].iloc[0])
from tensorflow.keras.preprocessing import image
img = image.load_img(sample_img_path, target_size=IMAGE_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

last_conv_layer_name = None
for layer in base_model.layers[::-1]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer_name = layer.name
        break
if last_conv_layer_name is None:
    raise ValueError("No convolutional layer found in the base model.")

heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
display_gradcam(sample_img_path, heatmap)


RUN 3

In [ ]:
# ----- STEP 0: Mount Google Drive -----
from google.colab import drive
drive.mount('/content/drive')

# ----- STEP 1: Import Libraries and Set Mixed Precision Policy -----
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.mixed_precision import set_global_policy

set_global_policy('mixed_float16')

# ----- STEP 2: Define Directory Paths and Load CSVs -----
train_img_dir = '/content/drive/My Drive/aptos2019-blindness-detection/train_images/'
test_img_dir  = '/content/drive/My Drive/aptos2019-blindness-detection/test_images/'
train_csv_path = '/content/drive/My Drive/aptos2019-blindness-detection/train.csv'
test_csv_path  = '/content/drive/My Drive/aptos2019-blindness-detection/test.csv'

train_df = pd.read_csv(train_csv_path)
test_df  = pd.read_csv(test_csv_path)

train_df['diagnosis'] = train_df['diagnosis'].astype(str)
train_df['filename'] = train_df['id_code'] + '.png'
test_df['filename']  = test_df['id_code'] + '.png'

# ----- STEP 3: Split Data into Training and Validation Sets -----
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
train_df['filename'] = train_df['id_code'] + '.png'
val_df['filename']   = val_df['id_code'] + '.png'
train_df['diagnosis'] = train_df['diagnosis'].astype(str)
val_df['diagnosis']   = val_df['diagnosis'].astype(str)

# ----- STEP 4: Define Image Size and Create Data Generators -----
IMAGE_SIZE = (380, 380)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='constant',
    cval=0
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_img_dir,
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    validate_filenames=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_img_dir,  # Assuming validation images are in the same folder
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

# ----- STEP 5: Set Up Callbacks with ModelCheckpoint -----
checkpoint_filepath = '/content/drive/My Drive/aptos2019-blindness-detection/best_model.keras'
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False
)
early_stopping_callback = EarlyStopping(patience=10, restore_best_weights=True)

callbacks = [checkpoint_callback, reduce_lr, early_stopping_callback]

# ----- STEP 6: Build the Model Using EfficientNetB4 and Unfreeze All Layers -----
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
base_model.trainable = True  # Unfreeze all layers for full fine-tuning

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
output = Dense(5, activation='softmax', dtype='float32')(x)

model = Model(inputs=base_model.input, outputs=output)
model.summary()

# ----- STEP 7: Compile the Model with a Lower Learning Rate for Fine-Tuning -----
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy', AUC(), Precision(), Recall()]
)

# ----- STEP 8: Train the Model (Full Fine-Tuning) -----
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks
)

# ----- STEP 9: Grad-CAM for Explainability -----
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    max_val = tf.math.reduce_max(heatmap)
    if max_val == 0:
        heatmap = tf.zeros_like(heatmap)
    else:
        heatmap = tf.maximum(heatmap, 0) / max_val
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    plt.figure(figsize=(8, 8))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# ----- STEP 10: Demonstrate Grad-CAM on a Sample Image -----
sample_img_path = os.path.join(train_img_dir, train_df['filename'].iloc[0])
from tensorflow.keras.preprocessing import image
img = image.load_img(sample_img_path, target_size=IMAGE_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

last_conv_layer_name = None
for layer in base_model.layers[::-1]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer_name = layer.name
        break
if last_conv_layer_name is None:
    raise ValueError("No convolutional layer found in the base model.")

heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
display_gradcam(sample_img_path, heatmap)


In [ ]:
# ----- STEP 0: Mount Google Drive -----
from google.colab import drive
drive.mount('/content/drive')

# ----- STEP 1: Import Libraries and Set Mixed Precision Policy -----
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.mixed_precision import set_global_policy

set_global_policy('mixed_float16')

# ----- STEP 2: Define Directory Paths and Load CSVs -----
train_img_dir = '/content/drive/My Drive/aptos2019-blindness-detection/train_images/'
test_img_dir  = '/content/drive/My Drive/aptos2019-blindness-detection/test_images/'
train_csv_path = '/content/drive/My Drive/aptos2019-blindness-detection/train.csv'
test_csv_path  = '/content/drive/My Drive/aptos2019-blindness-detection/test.csv'

train_df = pd.read_csv(train_csv_path)
test_df  = pd.read_csv(test_csv_path)

train_df['diagnosis'] = train_df['diagnosis'].astype(str)
train_df['filename'] = train_df['id_code'] + '.png'
test_df['filename']  = test_df['id_code'] + '.png'

# ----- STEP 3: Split Data into Training and Validation Sets -----
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
train_df['filename'] = train_df['id_code'] + '.png'
val_df['filename']   = val_df['id_code'] + '.png'
train_df['diagnosis'] = train_df['diagnosis'].astype(str)
val_df['diagnosis']   = val_df['diagnosis'].astype(str)

# ----- STEP 4: Define Image Size and Create Data Generators -----
IMAGE_SIZE = (380, 380)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='constant',
    cval=0
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_img_dir,
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    validate_filenames=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_img_dir,  # Assuming validation images are in the same folder
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

# ----- STEP 5: Set Up Callbacks with ModelCheckpoint -----
checkpoint_filepath = '/content/drive/My Drive/aptos2019-blindness-detection/best_model.keras'
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False
)
early_stopping_callback = EarlyStopping(patience=10, restore_best_weights=True)

callbacks = [checkpoint_callback, reduce_lr, early_stopping_callback]

# ----- STEP 6: Build the Model Using EfficientNetB4 -----
# Initially, freeze the base model for Phase 1 training.
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
base_model.trainable = False  # Freeze all layers initially

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
output = Dense(5, activation='softmax', dtype='float32')(x)

model = Model(inputs=base_model.input, outputs=output)
model.summary()

# ----- STEP 7: Compile the Model for Phase 1 (Frozen Base) -----
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy', AUC(), Precision(), Recall()]
)

# ----- STEP 8: Train the Model (Phase 1) -----
print("Starting Phase 1 Training (Frozen Base)...")
history_phase1 = model.fit(
    train_generator,
    epochs=10,  # Train initially for 10 epochs
    validation_data=val_generator,
    callbacks=callbacks
)

# ----- STEP 9: Progressive Unfreezing (Phase 2) -----
# Unfreeze the last 30 layers of the base model for fine-tuning.
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning.
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy', AUC(), Precision(), Recall()]
)

print("Starting Phase 2 Training (Unfrozen Last 30 Layers)...")
history_phase2 = model.fit(
    train_generator,
    epochs=10,  # Fine-tune for an additional 10 epochs
    validation_data=val_generator,
    callbacks=callbacks
)

# ----- STEP 10: Grad-CAM for Explainability -----
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    max_val = tf.math.reduce_max(heatmap)
    if max_val == 0:
        heatmap = tf.zeros_like(heatmap)
    else:
        heatmap = tf.maximum(heatmap, 0) / max_val
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    plt.figure(figsize=(8, 8))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# ----- STEP 11: Demonstrate Grad-CAM on a Sample Image -----
sample_img_path = os.path.join(train_img_dir, train_df['filename'].iloc[0])
from tensorflow.keras.preprocessing import image
img = image.load_img(sample_img_path, target_size=IMAGE_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

last_conv_layer_name = None
for layer in base_model.layers[::-1]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer_name = layer.name
        break
if last_conv_layer_name is None:
    raise ValueError("No convolutional layer found in the base model.")

heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
display_gradcam(sample_img_path, heatmap)


Runtime got interrupted


In [ ]:
from tensorflow.keras.models import load_model

# Load the last saved checkpoint
model = load_model('/content/drive/My Drive/aptos2019-blindness-detection/best_model.keras')

# Resume training from the current state (optionally set initial_epoch if you want to log from where you left off)
history_resume = model.fit(
    train_generator,
    epochs=10,  # set additional epochs
    validation_data=val_generator,
    callbacks=callbacks,
    initial_epoch=8  # if you want to indicate that 8 epochs have already been run
)


In [ ]:
#####################################################
# Phase 2: Progressive Fine-Tuning (Unfreeze Layers)
#####################################################

from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

# Load your best checkpoint from Phase 1
model = load_model('/content/drive/My Drive/aptos2019-blindness-detection/best_model.keras')

# Attempt to locate the base model (EfficientNet) within the loaded model.
base_model = None
for layer in model.layers:
    if "efficientnet" in layer.name.lower():
        base_model = layer
        break

if base_model is None:
    print("Warning: Base model not found in model layers. Unfreezing the entire model.")
    # Unfreeze the entire model as a fallback.
    model.trainable = True
else:
    # Unfreeze only the last 20 layers of the base model.
    for layer in base_model.layers[-20:]:
        layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning.
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'AUC', 'Precision', 'Recall']
)

# Continue training (fine-tuning) the model.
history_finetune = model.fit(
    train_generator,
    epochs=5,  # Adjust the number of epochs for fine-tuning as needed.
    validation_data=val_generator,
    callbacks=[checkpoint_callback, reduce_lr, early_stopping_callback]
)

#####################################################
# End of Phase 2 Fine-Tuning Script
#####################################################


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/drive/My Drive/aptos2019-blindness-detection/best_model.keras')

# Evaluate the model on your validation set
val_results = model.evaluate(val_generator, verbose=1)
print("Validation Results:", val_results)


In [ ]:
import numpy as np

# Assume 'history' is your training history dictionary (e.g., history.history)
# If you saved it to a pickle file, you can load it:
# import pickle
# with open('training_history.pkl', 'rb') as f:
#     history = pickle.load(f)

def extract_best_metrics(history):
    # Convert lists to numpy arrays for easier processing.
    acc = np.array(history.get('accuracy', []))
    val_acc = np.array(history.get('val_accuracy', []))
    loss = np.array(history.get('loss', []))
    val_loss = np.array(history.get('val_loss', []))
    auc = np.array(history.get('AUC', []))
    val_auc = np.array(history.get('val_AUC', []))
    precision = np.array(history.get('Precision', []))
    val_precision = np.array(history.get('val_Precision', []))
    recall = np.array(history.get('Recall', []))
    val_recall = np.array(history.get('val_Recall', []))

    # Find the best epoch based on validation accuracy.
    best_epoch = np.argmax(val_acc)

    metrics = {
        'best_epoch': int(best_epoch + 1),
        'train_accuracy': acc[best_epoch] if acc.size > 0 else None,
        'val_accuracy': val_acc[best_epoch] if val_acc.size > 0 else None,
        'train_loss': loss[best_epoch] if loss.size > 0 else None,
        'val_loss': val_loss[best_epoch] if val_loss.size > 0 else None,
        'train_auc': auc[best_epoch] if auc.size > 0 else None,
        'val_auc': val_auc[best_epoch] if val_auc.size > 0 else None,
        'train_precision': precision[best_epoch] if precision.size > 0 else None,
        'val_precision': val_precision[best_epoch] if val_precision.size > 0 else None,
        'train_recall': recall[best_epoch] if recall.size > 0 else None,
        'val_recall': val_recall[best_epoch] if val_recall.size > 0 else None,
    }
    return metrics

# Extract and print the best metrics.
best_metrics = extract_best_metrics(history)
print("Best Metrics from Training History:")
for k, v in best_metrics.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

# Check if the best validation accuracy is above 90%
if best_metrics.get('val_accuracy', 0) >= 0.90:
    print("Validation accuracy is above 90%.")
else:
    print("Validation accuracy is below 90%. Further fine-tuning is recommended.")


In [ ]:
#####################################################
# Full Integrated Script with Class Weights & Fine-Tuning
#####################################################

# ----- STEP 0: Mount Google Drive -----
from google.colab import drive
drive.mount('/content/drive')

# ----- STEP 1: Import Libraries and Set Mixed Precision Policy -----
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.mixed_precision import set_global_policy

set_global_policy('mixed_float16')

# ----- STEP 2: Define Directory Paths and Load CSVs -----
train_img_dir = '/content/drive/My Drive/aptos2019-blindness-detection/train_images/'
test_img_dir  = '/content/drive/My Drive/aptos2019-blindness-detection/test_images/'
train_csv_path = '/content/drive/My Drive/aptos2019-blindness-detection/train.csv'
test_csv_path  = '/content/drive/My Drive/aptos2019-blindness-detection/test.csv'

train_df = pd.read_csv(train_csv_path)
test_df  = pd.read_csv(test_csv_path)

train_df['diagnosis'] = train_df['diagnosis'].astype(str)
train_df['filename'] = train_df['id_code'] + '.png'
test_df['filename']  = test_df['id_code'] + '.png'

# ----- STEP 3: Split Data into Training and Validation Sets -----
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
train_df['filename'] = train_df['id_code'] + '.png'
val_df['filename']   = val_df['id_code'] + '.png'
train_df['diagnosis'] = train_df['diagnosis'].astype(str)
val_df['diagnosis']   = val_df['diagnosis'].astype(str)

# ----- STEP 4: Define Image Size and Create Data Generators -----
IMAGE_SIZE = (380, 380)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='constant',
    cval=0
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_img_dir,
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    validate_filenames=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_img_dir,  # Validation images are assumed to be in the same folder
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

# ----- STEP 5: Compute Class Weights for Imbalanced Data -----
# Convert diagnosis labels to integers for computing weights.
train_labels = train_df['diagnosis'].astype(int).values
classes = np.unique(train_labels)
class_weights = compute_class_weight('balanced', classes=classes, y=train_labels)
class_weight_dict = {int(c): float(w) for c, w in zip(classes, class_weights)}
print("Computed Class Weights:", class_weight_dict)

# ----- STEP 6: Set Up Callbacks with ModelCheckpoint -----
checkpoint_filepath = '/content/drive/My Drive/aptos2019-blindness-detection/best_model.keras'
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False
)
early_stopping_callback = EarlyStopping(patience=10, restore_best_weights=True)
callbacks = [checkpoint_callback, reduce_lr, early_stopping_callback]

# ----- STEP 7: Build the Model Using EfficientNetB4 with Full Fine-Tuning -----
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
base_model.trainable = True  # Unfreeze all layers for full fine-tuning

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
output = Dense(5, activation='softmax', dtype='float32')(x)

model = Model(inputs=base_model.input, outputs=output)
model.summary()

# ----- STEP 8: Compile the Model with a Lower Learning Rate for Fine-Tuning -----
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy', AUC(), Precision(), Recall()]
)

# ----- STEP 9: Train the Model (Full Fine-Tuning) with Class Weights -----
history = model.fit(
    train_generator,
    epochs=20,  # Increase or adjust epochs as needed
    validation_data=val_generator,
    callbacks=callbacks,
    class_weight=class_weight_dict  # Apply class weights to balance training
)

# ----- STEP 10: Grad-CAM for Explainability -----
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    max_val = tf.math.reduce_max(heatmap)
    if max_val == 0:
        heatmap = tf.zeros_like(heatmap)
    else:
        heatmap = tf.maximum(heatmap, 0) / max_val
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    plt.figure(figsize=(8, 8))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# ----- STEP 11: Demonstrate Grad-CAM on a Sample Image -----
sample_img_path = os.path.join(train_img_dir, train_df['filename'].iloc[0])
from tensorflow.keras.preprocessing import image
img = image.load_img(sample_img_path, target_size=IMAGE_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

last_conv_layer_name = None
for layer in base_model.layers[::-1]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer_name = layer.name
        break
if last_conv_layer_name is None:
    raise ValueError("No convolutional layer found in the base model.")

heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
display_gradcam(sample_img_path, heatmap)


In [ ]:
#####################################################
# Full Script: Load Randomized Model & Fine-Tune on Minimal Preprocessing Data
#####################################################

# ----- STEP 0: Mount Google Drive -----
from google.colab import drive
drive.mount('/content/drive')

# ----- STEP 1: Import Libraries and Set Mixed Precision Policy -----
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.mixed_precision import set_global_policy

# Enable mixed precision for faster training
set_global_policy('mixed_float16')

# ----- STEP 2: Define Directory Paths and Load CSVs -----
train_img_dir = '/content/drive/My Drive/aptos2019-blindness-detection/train_images/'
test_img_dir  = '/content/drive/My Drive/aptos2019-blindness-detection/test_images/'
train_csv_path = '/content/drive/My Drive/aptos2019-blindness-detection/train.csv'
test_csv_path  = '/content/drive/My Drive/aptos2019-blindness-detection/test.csv'

train_df = pd.read_csv(train_csv_path)
test_df  = pd.read_csv(test_csv_path)

train_df['diagnosis'] = train_df['diagnosis'].astype(str)
train_df['filename'] = train_df['id_code'] + '.png'
test_df['filename']  = test_df['id_code'] + '.png'

# ----- STEP 3: Split Data into Training and Validation Sets -----
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
train_df['filename'] = train_df['id_code'] + '.png'
val_df['filename']   = val_df['id_code'] + '.png'
train_df['diagnosis'] = train_df['diagnosis'].astype(str)
val_df['diagnosis']   = val_df['diagnosis'].astype(str)

# ----- STEP 4: Create Data Generators with Minimal Preprocessing (No Random Augmentation) -----
IMAGE_SIZE = (380, 380)

# Minimal preprocessing: only rescale images
minimal_train_datagen = ImageDataGenerator(rescale=1./255)
minimal_val_datagen = ImageDataGenerator(rescale=1./255)

minimal_train_generator = minimal_train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_img_dir,
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    validate_filenames=True
)

minimal_val_generator = minimal_val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_img_dir,
    x_col='filename',
    y_col='diagnosis',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

# ----- STEP 5: Compute Class Weights (if needed) -----
train_labels = train_df['diagnosis'].astype(int).values
classes = np.unique(train_labels)
class_weights = compute_class_weight('balanced', classes=classes, y=train_labels)
class_weight_dict = {int(c): float(w) for c, w in zip(classes, class_weights)}
print("Computed Class Weights:", class_weight_dict)

# ----- STEP 6: Load the Previously Trained Model -----
# This model was trained with heavy random augmentation
model = load_model('/content/drive/My Drive/aptos2019-blindness-detection/best_model.keras')
print("Model loaded from checkpoint.")

# ----- STEP 7: Re-Compile the Model for Fine-Tuning -----
# Here we use a lower learning rate for fine-tuning on minimal preprocessing data.
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy', AUC(), Precision(), Recall()]
)

# ----- STEP 8: Set Up Callbacks (Optional: Update filepath if desired) -----
checkpoint_filepath = '/content/drive/My Drive/aptos2019-blindness-detection/best_model_minimal.keras'
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False
)
early_stopping_callback = EarlyStopping(patience=10, restore_best_weights=True)
callbacks = [checkpoint_callback, reduce_lr, early_stopping_callback]

# ----- STEP 9: Fine-Tune the Model on Minimal Preprocessing Data -----
history_minimal = model.fit(
    minimal_train_generator,
    epochs=20,  # Adjust epochs as needed
    validation_data=minimal_val_generator,
    callbacks=callbacks,
    class_weight=class_weight_dict  # Use class weights to balance the training if needed
)

# ----- STEP 10: (Optional) Grad-CAM for Explainability -----
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    max_val = tf.math.reduce_max(heatmap)
    if max_val == 0:
        heatmap = tf.zeros_like(heatmap)
    else:
        heatmap = tf.maximum(heatmap, 0) / max_val
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    plt.figure(figsize=(8, 8))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# Demonstrate Grad-CAM on a Sample Image
sample_img_path = os.path.join(train_img_dir, train_df['filename'].iloc[0])
img = image.load_img(sample_img_path, target_size=IMAGE_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

last_conv_layer_name = None
for layer in model.layers[::-1]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer_name = layer.name
        break
if last_conv_layer_name is None:
    raise ValueError("No convolutional layer found in the model.")

heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
display_gradcam(sample_img_path, heatmap)
